# Learning

In [28]:
from collections import Counter

In [56]:
def interpret(text: str):
    lines = text.splitlines()
    rulesLines = lines[2:]
    rulesList = [line.split(' -> ') for line in rulesLines]
    rules = dict(rulesList)
    return lines[0],rules

def apply(input: str, rules: dict[str, str]):
    output = str("")
    for i in range(len(input)):
        curr = input[i:i+2]
        if curr in rules:
            output += input[i] + rules[curr]
        else:
            output += input[i]
    return output

def disparity(haps:Counter[str]):
    low, hi = ['',float('inf')], ['',float('-inf')]
    for k, v in haps.items():
        low = [k,v] if v<low[1] else low
        hi = [k,v] if v>hi[1] else hi
    return hi[1]-low[1]

def disparityStr(value: str):
    haps = Counter(value)
    return disparity(haps)


In [57]:
sample = """NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C"""

sampleOutput = """NCNBCHB
NBCCNBBBCBHCB
NBBBCNCCNBBNBNBBCHBHHBCHB
NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB""".splitlines()

In [58]:
template, rules = interpret(sample)

output = []
value = template
for i in range(10):
    value = apply(value, rules)
    output.append(value)

for i in range(len(sampleOutput)):
    if output[i] != sampleOutput[i]:
        print("mismatch", output[i], sampleOutput[i])

result = disparityStr(value)
assert(result == 1588)

Day 1

In [60]:
from aoc import read_data

template, rules = interpret(read_data('data/2021-14.txt'))

value = template
for i in range(10):
    value = apply(value, rules)

day1 = disparityStr(value)
assert(day1 == 3284)

Day 2

In [67]:
from collections import defaultdict

template, rules = interpret(sample)

pairs = Counter([template[i:i+2] for i in range(len(template)-1)])

def apply(current: Counter[str]): 
    new = defaultdict(lambda:0)
    for x,c in current.items():
        if x in rules:
            new[x[0]+rules[x]] += c
            new[rules[x]+x[1]] += c
        else:
            new[x] += c
    return Counter(new)

value = pairs
for i in range(40):
    value.update(apply(value))

first = defaultdict(lambda:0)
second = defaultdict(lambda:0)
for k,v in value.items():
    first[k[0]] += v
    second[k[1]] += v

# abbc -> ab bb bc
# first = a1 b2
# second = b2 c1
# first-second = a1
# second-first = c1
# first & second = b4
# first & second / 2 = b2

first = defaultdict(lambda:0,Counter({'a':1,'b':2}))
second = defaultdict(lambda:0,Counter({'b':2,'c':1}))

firstMinusSecond = [[k, v-second[k]] for k,v in first.items() if v-second[k]>0]
secondMinusFirst = [[k, v-first[k]] for k,v in second.items() if v-first[k]>0]
both = []

day2 = disparity(first)
